<a href="https://colab.research.google.com/github/2256haradityam/Projects/blob/main/CARPRICEPREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('https://github.com/2256haradityam/dataset/raw/refs/heads/main/breast-cancer.csv')

In [67]:
df.head()

,CLASSIFICATION,AGE,MENOPAUSE,SIZEOFTUMOR,INVNODES,NC,DM,BS,BQ,IR
0,0,0,0,2,0,0,3,0,0,0
1,0,1,0,4,0,0,2,1,2,0
2,0,1,0,4,0,0,2,0,0,0
3,0,3,1,5,0,0,2,1,1,0
4,0,1,0,1,0,0,2,1,3,0


In [58]:
df['SIZEOFTUMOR'] = df['SIZEOFTUMOR'].apply(lambda x: 0 if x in['45-49','5-9'] else (1 if x in ['50-54','0-4'] else (2 if x == '30-34' else (3 if x == '25-29' else (4 if x == '20-24' else (5 if x == '15-19' else(6 if x == '10-14' else (7 if x == '40-44' else 8))))))))

In [74]:
df['IR'].value_counts()

,count
IR,
0,217
1,68


In [82]:
columns_to_encode = ['AGE', 'SIZEOFTUMOR', 'MENOPAUSE', 'INVNODES', 'DM', 'BQ']

# Perform One-Hot Encoding and drop the original columns
df_encoded = pd.get_dummies(df, columns=columns_to_encode, drop_first=True)


In [84]:
df_encoded = df_encoded.astype(int)

In [86]:
df_encoded.head()

,CLASSIFICATION,NC,BS,IR,AGE_1,AGE_2,AGE_3,AGE_4,SIZEOFTUMOR_1,SIZEOFTUMOR_2,...,MENOPAUSE_1,MENOPAUSE_2,INVNODES_1,INVNODES_2,DM_2,DM_3,BQ_1,BQ_2,BQ_3,BQ_4
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,1,0,1,0,0,0
4,0,0,1,0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0


In [28]:
df['INVNODES'] = df['INVNODES'].apply(lambda x: 0 if x in ['0-2'] else (1 if x in ['3-5', '6-8', '9-11'] else (2 if x in ['15-17', '12-14', '24-26'] else x)))

In [42]:
df['BQ'] = df['BQ'].map({'left_low':0,'left_up':1,'right_up':2,'right_low':3,'central':4})

In [23]:
df['MENOPAUSE']=df['MENOPAUSE'].fillna(2.0)

In [61]:

from sklearn.metrics.pairwise import cosine_similarity

# Assuming you have a DataFrame named 'df'

# Filter rows with NA values in 'NC' column
df_na_nc = df[df['NC'].isna()]

# Remove rows with NA values in 'NC' column from the rest of the DataFrame
df_no_na_nc = df[df['NC'].notna()]

# Calculate cosine similarity between rows with NA values and rows without NA values
similarity_matrix = cosine_similarity(df_no_na_nc.drop('NC', axis=1), df_na_nc.drop('NC', axis=1))

# Find the index of the most similar row for each row with NA values
most_similar_index = similarity_matrix.argmax(axis=1)

# Get the corresponding rows from the DataFrame without NA values
similar_rows = df_no_na_nc.iloc[most_similar_index]

print(similar_rows)

    CLASSIFICATION  AGE  MENOPAUSE  SIZEOFTUMOR  INVNODES   NC  DM  BS  BQ  IR
4                0    1          0            1         0  0.0   2   1   3   0
0                0    0          0            2         0  0.0   3   0   0   0
7                0    3          1            4         0  0.0   1   0   0   0
6                0    2          0            3         0  0.0   2   0   0   0
0                0    0          0            2         0  0.0   3   0   0   0
..             ...  ...        ...          ...       ...  ...  ..  ..  ..  ..
0                0    0          0            2         0  0.0   3   0   0   0
0                0    0          0            2         0  0.0   3   0   0   0
6                0    2          0            3         0  0.0   2   0   0   0
4                0    1          0            1         0  0.0   2   1   3   0
4                0    1          0            1         0  0.0   2   1   3   0

[277 rows x 10 columns]


In [64]:
df[df['NC'].isna()]

,CLASSIFICATION,AGE,MENOPAUSE,SIZEOFTUMOR,INVNODES,NC,DM,BS,BQ,IR


In [ ]:
df['NC'].value

In [65]:
df['NC'] = df['NC'].astype(int)

In [87]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Assume df_encoded is your encoded DataFrame with the target column 'target'
X = df_encoded.drop(columns='CLASSIFICATION')
y = df_encoded['CLASSIFICATION']

# Standardize the features before applying PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Brute force PCA (testing different numbers of components)
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# Perform Gini index-based feature selection using RandomForestClassifier
rf_selector = RandomForestClassifier(n_estimators=100, criterion='gini', n_jobs=-1)
rf_selector.fit(X_pca, y)

# Select important features based on Gini index
model = SelectFromModel(rf_selector, prefit=True)
X_selected = model.transform(X_pca)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Classifiers and parameter grids for brute-force optimization
classifiers = {
    'SVM': SVC(),
    'RandomForest': RandomForestClassifier(n_jobs=-1),
    'KNN': KNeighborsClassifier(n_jobs=-1),
    'LogisticRegression': LogisticRegression(n_jobs=-1, max_iter=1000)
}

# Brute force parameter grid for each classifier
param_grids = {
    'SVM': {
        'C': [0.1, 1, 10, 100],
        'kernel': ['linear', 'rbf', 'poly'],
        'gamma': ['scale', 'auto']
    },
    'RandomForest': {
        'n_estimators': [100, 200, 500],
        'max_depth': [10, 20, 30, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'KNN': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'p': [1, 2]
    },
    'LogisticRegression': {
        'C': [0.1, 1, 10, 100],
        'solver': ['liblinear', 'lbfgs']
    }
}

# Brute force search using GridSearchCV
best_models = {}
for classifier_name, classifier in classifiers.items():
    print(f"Training {classifier_name}...")
    grid_search = GridSearchCV(classifier, param_grids[classifier_name], cv=5, n_jobs=-1, verbose=1)
    grid_search.fit(X_train, y_train)
    best_models[classifier_name] = grid_search.best_estimator_

# Evaluate each model
for model_name, model in best_models.items():
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name} Accuracy: {accuracy}")


Training SVM...
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Training RandomForest...
Fitting 5 folds for each of 108 candidates, totalling 540 fits
Training KNN...
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Training LogisticRegression...
Fitting 5 folds for each of 8 candidates, totalling 40 fits
SVM Accuracy: 0.7017543859649122
RandomForest Accuracy: 0.7017543859649122
KNN Accuracy: 0.6842105263157895
LogisticRegression Accuracy: 0.7017543859649122


In [88]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Assume df_encoded is your encoded DataFrame with the target column 'target'
X = df.drop(columns='CLASSIFICATION')
y = df['CLASSIFICATION']

# Standardize the features before applying PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Brute force PCA (testing different numbers of components)
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# Perform Gini index-based feature selection using RandomForestClassifier
rf_selector = RandomForestClassifier(n_estimators=100, criterion='gini', n_jobs=-1)
rf_selector.fit(X_pca, y)

# Select important features based on Gini index
model = SelectFromModel(rf_selector, prefit=True)
X_selected = model.transform(X_pca)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Classifiers and parameter grids for brute-force optimization
classifiers = {
    'SVM': SVC(),
    'RandomForest': RandomForestClassifier(n_jobs=-1),
    'KNN': KNeighborsClassifier(n_jobs=-1),
    'LogisticRegression': LogisticRegression(n_jobs=-1, max_iter=1000)
}

# Brute force parameter grid for each classifier
param_grids = {
    'SVM': {
        'C': [0.1, 1, 10, 100],
        'kernel': ['linear', 'rbf', 'poly'],
        'gamma': ['scale', 'auto']
    },
    'RandomForest': {
        'n_estimators': [100, 200, 500],
        'max_depth': [10, 20, 30, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'KNN': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'p': [1, 2]
    },
    'LogisticRegression': {
        'C': [0.1, 1, 10, 100],
        'solver': ['liblinear', 'lbfgs']
    }
}

# Brute force search using GridSearchCV
best_models = {}
for classifier_name, classifier in classifiers.items():
    print(f"Training {classifier_name}...")
    grid_search = GridSearchCV(classifier, param_grids[classifier_name], cv=5, n_jobs=-1, verbose=1)
    grid_search.fit(X_train, y_train)
    best_models[classifier_name] = grid_search.best_estimator_

# Evaluate each model
for model_name, model in best_models.items():
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name} Accuracy: {accuracy}")


Training SVM...
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Training RandomForest...
Fitting 5 folds for each of 108 candidates, totalling 540 fits
Training KNN...
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Training LogisticRegression...
Fitting 5 folds for each of 8 candidates, totalling 40 fits
SVM Accuracy: 0.7543859649122807
RandomForest Accuracy: 0.7017543859649122
KNN Accuracy: 0.6842105263157895
LogisticRegression Accuracy: 0.7192982456140351


In [89]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Assume df_encoded is your encoded DataFrame with the target column 'target'
X = df.drop(columns='CLASSIFICATION')
y = df['CLASSIFICATION']

# Standardize the features before applying PCA
#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X)

# Brute force PCA (testing different numbers of components)
pca = PCA()
X_pca = pca.fit_transform(X)

# Perform Gini index-based feature selection using RandomForestClassifier
rf_selector = RandomForestClassifier(n_estimators=100, criterion='gini', n_jobs=-1)
rf_selector.fit(X_pca, y)

# Select important features based on Gini index
model = SelectFromModel(rf_selector, prefit=True)
X_selected = model.transform(X_pca)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Classifiers and parameter grids for brute-force optimization
classifiers = {
    'SVM': SVC(),
    'RandomForest': RandomForestClassifier(n_jobs=-1),
    'KNN': KNeighborsClassifier(n_jobs=-1),
    'LogisticRegression': LogisticRegression(n_jobs=-1, max_iter=1000)
}

# Brute force parameter grid for each classifier
param_grids = {
    'SVM': {
        'C': [0.1, 1, 10, 100],
        'kernel': ['linear', 'rbf', 'poly'],
        'gamma': ['scale', 'auto']
    },
    'RandomForest': {
        'n_estimators': [100, 200, 500],
        'max_depth': [10, 20, 30, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'KNN': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'p': [1, 2]
    },
    'LogisticRegression': {
        'C': [0.1, 1, 10, 100],
        'solver': ['liblinear', 'lbfgs']
    }
}

# Brute force search using GridSearchCV
best_models = {}
for classifier_name, classifier in classifiers.items():
    print(f"Training {classifier_name}...")
    grid_search = GridSearchCV(classifier, param_grids[classifier_name], cv=5, n_jobs=-1, verbose=1)
    grid_search.fit(X_train, y_train)
    best_models[classifier_name] = grid_search.best_estimator_

# Evaluate each model
for model_name, model in best_models.items():
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name} Accuracy: {accuracy}")


Training SVM...
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Training RandomForest...
Fitting 5 folds for each of 108 candidates, totalling 540 fits
Training KNN...
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Training LogisticRegression...
Fitting 5 folds for each of 8 candidates, totalling 40 fits
SVM Accuracy: 0.7192982456140351
RandomForest Accuracy: 0.7017543859649122
KNN Accuracy: 0.7017543859649122
LogisticRegression Accuracy: 0.7017543859649122
